In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
import optuna

In [2]:
def lgb_objective(trial):
    params = {
        'n_iter'           : 200,
        'verbosity'        : -1,
        'objective'        : 'l1',
        'random_state'     : 42,
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'        : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'       : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 250),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X, y   = df_all.drop(columns=[target]), df_all[target]
        
    if SMAPE_ENABLED:
        y = to_percent(X, y)
    
    train_times = list(range(38))
    valid_times = [38]
    
    y_train = y[X['scale'].isin(train_times)]
    y_valid = y[X['scale'].isin(valid_times)]
    
    X_train = X[X['scale'].isin(train_times)]
    X_valid = X[X['scale'].isin(valid_times)]
    
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    
    if SMAPE_ENABLED:
        preds = from_percent(X_valid, preds)
        y_valid = from_percent(X_valid, y_valid)
    
    score = smape(y_valid, preds)
    
    return score

In [6]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
census = pd.read_csv('./census_starter.csv')

In [7]:
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100


In [13]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pct_bb_2017            3142 non-null   float64
 1   pct_bb_2018            3142 non-null   float64
 2   pct_bb_2019            3142 non-null   float64
 3   pct_bb_2020            3141 non-null   float64
 4   pct_bb_2021            3141 non-null   float64
 5   cfips                  3142 non-null   int64  
 6   pct_college_2017       3142 non-null   float64
 7   pct_college_2018       3142 non-null   float64
 8   pct_college_2019       3142 non-null   float64
 9   pct_college_2020       3141 non-null   float64
 10  pct_college_2021       3141 non-null   float64
 11  pct_foreign_born_2017  3142 non-null   float64
 12  pct_foreign_born_2018  3142 non-null   float64
 13  pct_foreign_born_2019  3142 non-null   float64
 14  pct_foreign_born_2020  3141 non-null   float64
 15  pct_

In [9]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [10]:
state_dict = train[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()

test['state'] = test['cfips'].map(state_dict['state'])
test['county'] = test['cfips'].map(state_dict['county'])

In [12]:
test

,row_id,cfips,first_day_of_month,state,county
0,1001_2022-11-01,1001,2022-11-01,Alabama,Autauga County
1,1003_2022-11-01,1003,2022-11-01,Alabama,Baldwin County
2,1005_2022-11-01,1005,2022-11-01,Alabama,Barbour County
3,1007_2022-11-01,1007,2022-11-01,Alabama,Bibb County
4,1009_2022-11-01,1009,2022-11-01,Alabama,Blount County
...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,Wyoming,Sweetwater County
25076,56039_2023-06-01,56039,2023-06-01,Wyoming,Teton County
25077,56041_2023-06-01,56041,2023-06-01,Wyoming,Uinta County
25078,56043_2023-06-01,56043,2023-06-01,Wyoming,Washakie County
